In [ ]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [ ]:
load_dotenv()
os.environ["OPENAI_API_TYPE"] = "OPENAI_API_TYPE"
os.environ["OPENAI_API_BASE"] = "OPENAI_API_BASE"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY" 
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model = "text-embedding"

In [ ]:
import os
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] =  "AZURE_COGNITIVE_SEARCH_SERVICE_NAME"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "AZURE_COGNITIVE_SEARCH_INDEX_NAME"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "AZURE_COGNITIVE_SEARCH_API_KEY"

In [ ]:
# Initialize gpt-35-turbo and our embedding model
llm = AzureChatOpenAI(deployment_name="gpt-35-4k")
embeddings = OpenAIEmbeddings(deployment='text-embedding')

In [ ]:
vector_store_address: str = ""
vector_store_password: str = ""

In [ ]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
index_name = "giveindexname"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("SampleData.pdf")
documents = loader.load_and_split()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

# Add documents to Azure Search
vector_store.add_documents(documents=docs)

In [ ]:
# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
"""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=vector_store.as_retriever(),
                                           return_source_documents=True,
                                           verbose=False,max_tokens_limit=4000)

In [ ]:
chat_history = []
query = "envestnet recent revenue"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])